In [32]:
import os
from pathlib import Path

from tensorboardX import SummaryWriter


from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.utils import common_utils

import datetime

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = {'cfg_file': '../cfgs/kitti_models/my_pointrcnn.yaml', 'batch_size': 1, 'epochs': None, 'workers': 1, 'extra_tag': 'default', 'ckpt': None, 'pretrained_model': None, 'launcher': 'none', 'tcp_port': 18888, 'sync_bn': False, 'fix_random_seed': False, 'ckpt_save_interval': 1, 'local_rank': 0, 'max_ckpt_save_num': 30, 'merge_all_iters_to_one_epoch': False, 'set_cfgs': None, 'max_waiting_mins': 0, 'start_epoch': 0, 'save_to_file': False}

args = Struct(**args)
print(args.cfg_file)


args.cfg_file= "./my_pointrcnn.yaml"
print(args.cfg_file)
cfg_from_yaml_file(args.cfg_file, cfg)
cfg.TAG = Path(args.cfg_file).stem
cfg.EXP_GROUP_PATH = '/'.join(args.cfg_file.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'
if args.set_cfgs is not None:
    cfg_from_list(args.set_cfgs, cfg)


if args.launcher == 'none':
    dist_train = False
    total_gpus = 1
else:
    total_gpus, cfg.LOCAL_RANK = getattr(common_utils, 'init_dist_%s' % args.launcher)(
        args.tcp_port, args.local_rank, backend='nccl'
    )
    dist_train = True

if args.batch_size is None:
    args.batch_size = cfg.OPTIMIZATION.BATCH_SIZE_PER_GPU
else:
    assert args.batch_size % total_gpus == 0, 'Batch size should match the number of gpus'
    args.batch_size = args.batch_size // total_gpus

args.epochs = cfg.OPTIMIZATION.NUM_EPOCHS if args.epochs is None else args.epochs

if args.fix_random_seed:
    common_utils.set_random_seed(666)

output_dir = cfg.ROOT_DIR / 'output' / cfg.EXP_GROUP_PATH / cfg.TAG / args.extra_tag
ckpt_dir = output_dir / 'ckpt'
output_dir.mkdir(parents=True, exist_ok=True)
ckpt_dir.mkdir(parents=True, exist_ok=True)

log_file = output_dir / ('log_train_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)

# log to file
logger.info('**********************Start logging**********************')
gpu_list = os.environ['CUDA_VISIBLE_DEVICES'] if 'CUDA_VISIBLE_DEVICES' in os.environ.keys() else 'ALL'
logger.info('CUDA_VISIBLE_DEVICES=%s' % gpu_list)

if dist_train:
    logger.info('total_batch_size: %d' % (total_gpus * args.batch_size))
for key, val in vars(args).items():
    logger.info('{:16} {}'.format(key, val))
log_config_to_file(cfg, logger=logger)
if cfg.LOCAL_RANK == 0:
    os.system('cp %s %s' % (args.cfg_file, output_dir))

tb_log = SummaryWriter(log_dir=str(output_dir / 'tensorboard')) if cfg.LOCAL_RANK == 0 else None

from pcdet.datasets import build_dataloader

train_set, train_loader, train_sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG,
        class_names=cfg.CLASS_NAMES,
        batch_size=args.batch_size,
        dist=dist_train, workers=args.workers,
        logger=logger,
        training=True,
        merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
        total_epochs=args.epochs
    )


2020-08-19 01:16:56,218   INFO  **********************Start logging**********************
2020-08-19 01:16:56,218   INFO  **********************Start logging**********************
2020-08-19 01:16:56,218   INFO  **********************Start logging**********************
2020-08-19 01:16:56,221   INFO  CUDA_VISIBLE_DEVICES=ALL
2020-08-19 01:16:56,221   INFO  CUDA_VISIBLE_DEVICES=ALL
2020-08-19 01:16:56,221   INFO  CUDA_VISIBLE_DEVICES=ALL
2020-08-19 01:16:56,224   INFO  cfg_file         ./my_pointrcnn.yaml
2020-08-19 01:16:56,224   INFO  cfg_file         ./my_pointrcnn.yaml
2020-08-19 01:16:56,224   INFO  cfg_file         ./my_pointrcnn.yaml
2020-08-19 01:16:56,227   INFO  batch_size       1
2020-08-19 01:16:56,227   INFO  batch_size       1
2020-08-19 01:16:56,227   INFO  batch_size       1
2020-08-19 01:16:56,230   INFO  epochs           80
2020-08-19 01:16:56,230   INFO  epochs           80
2020-08-19 01:16:56,230   INFO  epochs           80
2020-08-19 01:16:56,232   INFO  workers    

../cfgs/kitti_models/my_pointrcnn.yaml
./my_pointrcnn.yaml


In [33]:
train_set[0]


{'points': array([[ 1.8324e+01,  4.9000e-02,  8.2900e-01,  0.0000e+00],
       [ 1.8344e+01,  1.0600e-01,  8.2900e-01,  0.0000e+00],
       [ 5.1299e+01,  5.0500e-01,  1.9440e+00,  0.0000e+00],
       ...,
       [ 6.2720e+00, -4.0000e-02, -1.6370e+00,  3.0000e-01],
       [ 6.2740e+00, -3.1000e-02, -1.6370e+00,  3.1000e-01],
       [ 6.2760e+00, -1.1000e-02, -1.6380e+00,  3.1000e-01]],
      dtype=float32), 'frame_id': '000000', 'calib': <pcdet.utils.calibration_kitti.Calibration object at 0x7f420b783c18>, 'gt_names': array(['Pedestrian'], dtype='<U10'), 'gt_boxes': array([[ 8.73138  , -1.8559176, -0.6546994,  1.2      ,  0.48     ,
         1.89     , -1.5807964]], dtype=float32), 'road_plane': array([-0.00705174, -0.99977906, -0.0198015 ,  1.68036694])}
nnnnnnnnnnnn
{'points': array([[10.047032  , -6.464702  , -0.17514108,  0.        ],
       [10.072101  , -6.4367895 , -0.17514108,  0.07      ],
       [10.10466   , -6.4122066 , -0.17514108,  0.09      ],
       ...,
       [ 6.135

NameError: name 'mmmmm' is not defined